In [1]:
from imutils.video import VideoStream
import numpy as np
import dlib
import imutils
import time
import cv2

In [2]:
#defining prototext and caffemodel paths

caffeModel = "/Users/chauhanvarnika/Documents/Py-OpenCV/FaceDetection/Lip gesture Project/model/res10_300x300_ssd_iter_140000.caffemodel"

prototextPath = "/Users/chauhanvarnika/Documents/Py-OpenCV/FaceDetection/Lip gesture Project/model/deploy.prototxt.txt"

In [3]:
#defining dlib landmark detection path 

detFile = "/Users/chauhanvarnika/Documents/Py-OpenCV/FaceDetection/Lip gesture Project/model/shape_predictor_68_face_landmarks.dat"
getLandmark = dlib.shape_predictor(detFile)

In [4]:
#Load Model
print("Loading model...................")
net = cv2.dnn.readNetFromCaffe(prototextPath,caffeModel)

Loading model...................


In [5]:
def drawPoints(image, faceLandmarks, startpoint, endpoint, isClosed=True):
  for i in range(startpoint, endpoint+1):
    
    #For mouth open and close gesture
    lip_left_coord = (int (faceLandmarks[60][0]), int(faceLandmarks[60][1]))
    lip_right_coord = (int (faceLandmarks[64][0]), int(faceLandmarks[64][1]))
    lip_wd = (lip_left_coord[0] - lip_right_coord[0]) ** 2 +( lip_left_coord[1] - lip_right_coord[1]) ** 2 

    lip_top_coord = (int (faceLandmarks[62][0]), int(faceLandmarks[62][1]))
    lip_btm_coord = (int (faceLandmarks[66][0]), int(faceLandmarks[66][1]))
    lip_ht = (lip_top_coord[0] - lip_btm_coord[0]) ** 2 +( lip_top_coord[1] - lip_btm_coord[1]) ** 2
    
    #adding aspect ratio threshold for mouth open and close state
    aspect_ratio = round((lip_ht/lip_wd), 2)
    if aspect_ratio > 0.08:                                                                             
        cv2.putText(image, "open mouth", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (10,0,255))
    if aspect_ratio == 0.0:                                                                             
        cv2.putText(image, "close mouth", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (10,0,255))
  

In [6]:
def getFaceEmbedding(image, detections):
    for i in range(0, detections.shape[2]):
        
        confidence = detections[0, 0, i, 2]
        # filter detections by confidence greater than the minimum confidence
        if confidence < 0.5 :
            continue
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        dlib_rectangle = dlib.rectangle(left=startX, top=startY, right=endX, bottom=endY)
        
        #Adding confidence to the image
        #text = "{:.2f}%".format(confidence * 100)
        #y = startY - 10 if startY - 10 > 10 else startY + 10
        #cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

        #Getting landmarks by passing the frame to the landmark detector
        shape = getLandmark(image, dlib_rectangle)  
        
        #Creating a 68 X 2 numpy array to store (x,y) coordiates of each landmark.
        shape_np = np.zeros((68, 2), dtype="int")                                                       
        for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)
        shape = shape_np

        # Display the landmarks
        for i, (x, y) in enumerate(shape):
            if i == 47 :
                drawPoints(image, shape, 48, 59)
            elif i == 59:
                drawPoints(image, shape, 60, 67)
                

In [7]:
# initialize the video stream to get the live video frames
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

[INFO] starting video stream...


In [8]:
while True:
    frame = vs.read()
    frame = imutils.resize(frame,width=400)

    # extract the dimensions , Resize image into 300x300 and converting image into blobFromImage
    (h, w) = frame.shape[:2]
    # blobImage convert RGB (104.0, 177.0, 123.0)
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,(300, 300), (104.0, 177.0, 123.0))
   #passing blob through the network to detect and pridiction
    net.setInput(blob)
    detections = net.forward()
    
    getFaceEmbedding(frame, detections)
    
    # Display the image
    cv2.imshow('Lip Gesture Detection', frame)

    # escape button to terminate the code
    if cv2.waitKey(10) == 27:
        break